<a href="https://colab.research.google.com/github/amr-elbana/EcoSortAI-Classifier/blob/Preprocessing-amr/Data_Augmentation/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data_Augmentation**



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data_dir = "/content/drive/MyDrive/DL_Project/realWaste_Augmented2"
test_dir = "/content/drive/MyDrive/DL_Project/realWaste_Augmented_test2"

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
import os
import shutil

# Define paths
original_data_dir = "/content/drive/MyDrive/DL_Project/realWaste_Augmented"  # Update this to your local path
augmented_data_dir = "/content/drive/MyDrive/DL_Project/realWaste_Augmented2"  # Update this to your local path

# Create the augmented directory if it doesn't exist
os.makedirs(augmented_data_dir, exist_ok=True)

# Initialize ImageDataGenerator with augmentation options
datagen = ImageDataGenerator(
    rotation_range=30,  # Increased rotation for more variety
    width_shift_range=0.25,  # Slightly increased shifts
    height_shift_range=0.25,
    shear_range=0.25,
    zoom_range=0.3,  # Larger zoom variations
    horizontal_flip=True,
    vertical_flip=True,  # Include vertical flips
    brightness_range=(0.8, 1.2),  # Simulate lighting changes
    channel_shift_range=50,  # Adjust RGB channels
    fill_mode='nearest',
)

# Loop through each class (Healthy and Rotten)
for class_dir in os.listdir(original_data_dir):
    class_path = os.path.join(original_data_dir, class_dir)
    if os.path.isdir(class_path):
        save_path = os.path.join(augmented_data_dir, class_dir)
        os.makedirs(save_path, exist_ok=True)

        # Dynamically calculate augmentation factor for this class
        augmentation_factor = max(2, 1000 // len(os.listdir(class_path)))  # Ensure at least 5 augmentations per original

        # Copy original images to augmented directory
        for filename in os.listdir(class_path):
            original_img_path = os.path.join(class_path, filename)
            target_img_path = os.path.join(save_path, filename)
            shutil.copy(original_img_path, target_img_path)  # Copy original image

            # Load image for augmentation
            img = load_img(original_img_path)
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)

            # Generate augmented versions
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=save_path, save_prefix='aug', save_format='jpg'):
                i += 1
                if i >= augmentation_factor:
                    break  # Stop after creating the required number of augmentations

print("Augmentation complete. Augmented images are saved in:", augmented_data_dir)


Augmentation complete. Augmented images are saved in: /content/drive/MyDrive/DL_Project/realWaste_Augmented2


In [5]:
# Define paths
original_data_dir = "/content/drive/MyDrive/DL_Project/realWaste_Augmented_test"  # Update this to your local path
augmented_data_dir ="/content/drive/MyDrive/DL_Project/realWaste_Augmented_test2"  # Update this to your local path

# Create the augmented directory if it doesn't exist
os.makedirs(augmented_data_dir, exist_ok=True)

# Initialize ImageDataGenerator with augmentation options
datagen = ImageDataGenerator(
    rotation_range=30,  # Increased rotation for more variety
    width_shift_range=0.25,  # Slightly increased shifts
    height_shift_range=0.25,
    shear_range=0.25,
    zoom_range=0.3,  # Larger zoom variations
    horizontal_flip=True,
    vertical_flip=True,  # Include vertical flips
    brightness_range=(0.8, 1.2),  # Simulate lighting changes
    channel_shift_range=50,  # Adjust RGB channels
    fill_mode='nearest',
)

# Loop through each class (Healthy and Rotten)
for class_dir in os.listdir(original_data_dir):
    class_path = os.path.join(original_data_dir, class_dir)
    if os.path.isdir(class_path):
        save_path = os.path.join(augmented_data_dir, class_dir)
        os.makedirs(save_path, exist_ok=True)

        # Dynamically calculate augmentation factor for this class
        augmentation_factor = max(2, 1000 // len(os.listdir(class_path)))  # Ensure at least 2 augmentations per original

        # Copy original images to augmented directory
        for filename in os.listdir(class_path):
            original_img_path = os.path.join(class_path, filename)
            target_img_path = os.path.join(save_path, filename)
            shutil.copy(original_img_path, target_img_path)  # Copy original image

            # Load image for augmentation
            img = load_img(original_img_path)
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)

            # Generate augmented versions
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=save_path, save_prefix='aug', save_format='jpg'):
                i += 1
                if i >= augmentation_factor:
                    break  # Stop after creating the required number of augmentations

print("Augmentation complete. Augmented images are saved in:", augmented_data_dir)


Augmentation complete. Augmented images are saved in: /content/drive/MyDrive/DL_Project/realWaste_Augmented_test2
